In [1]:
#data = pd.read_parquet(df_path)
from functions_training_pipeline import import_data, data_normalization
import pandas as pd

In [2]:
df_path = r"../Data/combined/dataframe_extended/"
date_from = '2018-06-13'
date_to = '2018-06-17'

In [ ]:
data = import_data(date_from, date_to, df_path)
data = data_normalization(data)

In [6]:
import functions_training_pipeline_pyspark as f


In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LogisticRegressionGridSearch").getOrCreate()


from pyspark.ml.regression import LinearRegression
lr = f.Model(model=LinearRegression, name="LinearRegression")

your 131072x1 screen size is bogus. expect trouble
23/05/13 12:44:46 WARN Utils: Your hostname, LAPTOP-3TD5GQG7 resolves to a loopback address: 127.0.1.1; using 172.20.146.42 instead (on interface eth0)
23/05/13 12:44:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/13 12:44:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
hyperparameters_for_grid = {"fit_intercept": [True]}
lr.hyperparameters = lr.create_hyperparameter_grid(hyperparameters_for_grid)


In [10]:

columns = data.columns.drop(["opt_value"])
data_ = spark.createDataFrame(data)

In [11]:

lr.spatial_cv(data_, columns, target_normalized=True)

23/05/13 13:35:48 ERROR Instrumentation: java.lang.IllegalArgumentException: features does not exist. Available: x, y
	at org.apache.spark.sql.types.StructType.$anonfun$apply$1(StructType.scala:285)
	at scala.collection.immutable.Map$Map2.getOrElse(Map.scala:236)
	at org.apache.spark.sql.types.StructType.apply(StructType.scala:284)
	at org.apache.spark.ml.util.SchemaUtils$.checkColumnTypes(SchemaUtils.scala:59)
	at org.apache.spark.ml.util.SchemaUtils$.validateVectorCompatibleColumn(SchemaUtils.scala:205)
	at org.apache.spark.ml.clustering.KMeansParams.validateAndTransformSchema(KMeans.scala:121)
	at org.apache.spark.ml.clustering.KMeansParams.validateAndTransformSchema$(KMeans.scala:120)
	at org.apache.spark.ml.clustering.KMeans.validateAndTransformSchema(KMeans.scala:295)
	at org.apache.spark.ml.clustering.KMeans.transformSchema(KMeans.scala:599)
	at org.apache.spark.ml.PipelineStage.transformSchema(Pipeline.scala:71)
	at org.apache.spark.ml.clustering.KMeans.$anonfun$fit$1(KMeans.sc

IllegalArgumentException: features does not exist. Available: x, y

In [ ]:
class Model:
    def __init__(self, model, name):
        self.model = model
        self.hyperparameters = []  # list of dictionaries with hyperparameters
        self.name = name

    def create_hyperparameter_grid(self, hyperparameters):
        # # create a ParamGridBuilder object
        # param_grid = ParamGridBuilder()
        # # add hyperparameters and their possible values to the grid
        # for hyperparam, values in hyperparameters.items():
        #     param_grid = param_grid.addGrid(hyperparam, values)
        # # build the grid of hyperparameters
        # hyperparameters_grid = param_grid.build()
        # return hyperparameters_grid
        return ParamGridBuilder().build()

    def __kmeans_split(self, df, split_variable_name, plot=False, verbose=False):
        kmeans = KMeans(k=5, seed=0).fit(df.select("x", "y"))
        df = df.withColumn(split_variable_name, kmeans.transform(df.select("x", "y")).select("prediction")) # not sure about select prediction

        if verbose:
            df.groupBy(split_variable_name).count().show()

        if plot:
            # Convert Spark DataFrame to Pandas DataFrame for plotting
            pandas_df = df.select("x", "y", split_variable_name).toPandas()
            plt.scatter(pandas_df["x"], pandas_df["y"], c=pandas_df[split_variable_name], edgecolor="none", s=0.05)
            plt.show()
        return df

    def __train_test_split(self, df, columns, split_variable_name, split_index):
        train = df.filter(df[split_variable_name] != split_index)
        test = df.filter(df[split_variable_name] == split_index)
        train_X = train.select(columns)
        train_y = train.select("opt_value").rdd.flatMap(lambda x: x).collect()
        test_X = test.select(columns)
        test_y = test.select("opt_value").rdd.flatMap(lambda x: x).collect()
        return train_X, train_y, test_X, test_y

    def __tune_hyperparameters(self, df, columns, split_variable_name):
        """
        This function performs hyperparameter tuning in the inner loop of nested cross-validation.

        Args:
            df (pyspark.sql.DataFrame): DataFrame with data.

            columns (list): List with column names to be used in the model.

            split_variable_name (str): Name of column with k-means split.

        Returns:
            dict: Dictionary with best hyperparameters.
        """
        all_hyperparameter_scores = []
        for split in df.select(split_variable_name).distinct().rdd.flatMap(lambda x: x).collect():
            train_X, train_y, test_X, test_y = self.__train_test_split(df, columns, split_variable_name, split)
            one_loop_hyperparameter_scores = []
            if isinstance(self.hyperparameters, list):
                for hyperparams in self.hyperparameters:
                    regressor = self.model(**hyperparams)
                    evaluator = RegressionEvaluator(metricName="rmse")
                    param_grid = ParamGridBuilder().build()
                    crossval = CrossValidator(estimator=regressor, estimatorParamMaps=param_grid, evaluator=evaluator)
                    cv_model = crossval.fit(train_X)
                    predictions = cv_model.transform(test_X)
                    rmse = evaluator.evaluate(predictions)
                    one_loop_hyperparameter_scores.append(rmse)
            else:
                print("hyperparameters must be a list")
            all_hyperparameter_scores.append(one_loop_hyperparameter_scores)

        mean_hyperparameters = np.mean(all_hyperparameter_scores, axis=0)
        best_hyperparameters = self.hyperparameters[np.argmin(mean_hyperparameters)]
        return best_hyperparameters

    def __check_columns(self, columns):
        for col in columns:
            if col in ["row", "col", "date", "opt_value"]:
                print(f"Column {col} should not be included")
                assert False

    def spatial_cv(self, df, columns, target_normalized):
        self.__check_columns(columns)
        self.columns = columns

        rmse_list_train = []
        rmse_list_test = []
        r2_list_train = []
        r2_list_test = []
        self.cv_model_list = []

        # Split the data into outer folds
        df = self.__kmeans_split(df, "outer_area")

        # Create a SparkSession
        #spark = SparkSession.builder.getOrCreate()

        # df_spark = spark.createDataFrame(df)

        # For each outer fold
        for outer_split in df.select("outer_area").distinct().rdd.flatMap(lambda x: x).collect():
            print("Spatial CV, outer split:", outer_split)

            # Define only train set (to be used in the inner loop of nested cross-validation)
            train = df.filter(df["outer_area"] != outer_split)

            # Split the data into inner folds
            train = self.__kmeans_split(train, "inner_area")

            # Tune hyperparameters (all inner loops of nested cross-validation are executed in this function)
            best_hyperparam = self.__tune_hyperparameters(train, columns, split_variable_name="inner_area")

            # With the best hyperparameters, train the model on the outer fold
            train_X, train_y, test_X, test_y = self.__train_test_split(
                df, columns, split_variable_name="outer_area", split_index=outer_split
            )

            # Convert train and test data to Spark DataFrame
            train_data = spark.createDataFrame(train_X.rdd.zipWithIndex().map(lambda x: (x[1],) + x[0]).toDF())
            train_data = train_data.toDF(*["index"] + train_X.columns)
            train_data = train_data.withColumn("opt_value", lit(train_y))

            test_data = spark.createDataFrame(test_X.rdd.zipWithIndex().map(lambda x: (x[1],) + x[0]).toDF())
            test_data = test_data.toDF(*["index"] + test_X.columns)
            test_data = test_data.withColumn("opt_value", lit(test_y))

            # Create the model with the best hyperparameters
            regressor = self.model(**best_hyperparam)
            model = regressor.fit(train_data)
            self.cv_model_list.append(model)

            # Make predictions on train and test data
            train_y_predicted = model.transform(train_data).select("prediction").rdd.flatMap(lambda x: x).collect()
            test_y_predicted = model.transform(test_data).select("prediction").rdd.flatMap(lambda x: x).collect()

            if target_normalized:
                train_y_predicted = [np.exp(y) - 1 for y in train_y_predicted]
                test_y_predicted = [np.exp(y) - 1 for y in test_y_predicted]

            evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
            rmse_list_train = evaluator.evaluate(train_y_predicted)
            rmse_list_test = evaluator.evaluate(test_y_predicted)
            evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="r2")
            r2_list_train = evaluator.evaluate(train_y_predicted)
            r2_list_test = evaluator.evaluate(test_y_predicted)

        # Results
        self.rmse_train = np.mean(rmse_list_train)
        self.rmse_std_train = np.std(rmse_list_train)
        self.rmse_test = np.mean(rmse_list_test)
        self.rmse_std_test = np.std(rmse_list_test)
        self.r2_train = np.mean(r2_list_train)
        self.r2_std_train = np.std(r2_list_train)
        self.r2_test = np.mean(r2_list_test)
        self.r2_std_test = np.std(r2_list_test)

        # Find best hyperparameters for the WHOLE dataset (instead of only one fold at a time)
        df = self.__kmeans_split(df, "final_split_areas")
        for split in df.select("final_split_areas").distinct().rdd.flatMap(lambda x: x).collect():
            print("Spatial CV, final split:", split)
            final_hyperparameters = self.__tune_hyperparameters(df, columns, split_variable_name="final_split_areas")

        # Fit final model
        self.final_model = self.model(**final_hyperparameters)
        final_data = spark.createDataFrame(df.rdd.zipWithIndex().map(lambda x: (x[1],) + x[0]).toDF())
        final_data = final_data.toDF(*["index"] + df.columns)
        final_data = final_data.withColumn("opt_value", lit(df["opt_value"]))
        self.final_model = self.final_model.fit(final_data)

        return